In [1]:
import csv
import numpy as np

from typing import Tuple
from scipy.spatial.transform import Rotation as R

from utils import to_transformation_matrix, register, compute_errors

In [2]:
def extract_data(file_path: str) -> Tuple[np.ndarray, np.ndarray]:
    """
    Read csv file and generate Laser_point and Robot_T_tool0
    """
    file = open(file_path)
    csv_reader = csv.reader(file)

    _ = next(csv_reader)
    rows = []
    for row in csv_reader:
        row = [float(i) for i in row[1:]]
        rows.append(row)
    rows = np.array(rows)

    return rows[:, :3], rows[:, 3:]

In [3]:
# Comupted using TCP calibration with laser
tool0_T_SMR = np.array([
    [1, 0, 0, 0.68],
    [0, 1, 0, 1.83],
    [0, 0, 1, 28.76],
    [0, 0, 0, 1]
])

# Extracted from laser software
Laser_T_swing_chop = np.array([
    [-0.9999, -0.0082, 0.0079, 2513.5597],
    [0.0083, -1.0000, 0.0027, -270.4662],
    [0.0079, 0.0028, 1.0000, -366.2441],
    [0, 0, 0, 1]
])

swing_chop_T_joint1 = np.array([
    [-1, 0, 0, 0],
    [0, -1, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1]
])

# Extracted from UR calibration file
RobotBase_T_joint1 = np.eye(4)
RobotBase_T_joint1[:3, :3] = R.from_euler('xyz', [0, 0, 6.93481305019672e-08], degrees=False).as_matrix()
RobotBase_T_joint1[:3, 3] = [0, 0, 181.0117558098409]

# Transformation matrix from swing-chop method
Laser_T_RobotBase_current_method =  Laser_T_swing_chop @ swing_chop_T_joint1 @ np.linalg.inv(RobotBase_T_joint1)

In [4]:
Laser_points, RobotBase_T_tool0 = extract_data("data.csv")

RobotBase_T_tool0 = to_transformation_matrix(RobotBase_T_tool0)

RobotBase_T_SMR = np.matmul(RobotBase_T_tool0, tool0_T_SMR[None])
RobotBase_points = RobotBase_T_SMR[:, :3, 3]

In [5]:
# Compute error for swing chop method
error = compute_errors(RobotBase_points, Laser_points, Laser_T_RobotBase_current_method)
print("Current Method:")
print(f"Mean: {error.mean():0.2f}mm\t Max: {error.max():0.2f}mm \n")

Current Method:
Mean: 0.92mm	 Max: 1.85mm 



In [6]:
# Compute error for 3D point fitting method

# Split some data for optimization and remaining for testing
PARTITION = 45

error_mean, error_max = [], []

for i in range(25):
    idx = np.arange(60, dtype=int)
    np.random.seed(i)
    np.random.shuffle(idx)

    RobotBase_points_train, Laser_points_train = RobotBase_points[idx[:PARTITION]], Laser_points[idx[:PARTITION]]
    RobotBase_points_test, Laser_points_test = RobotBase_points[idx[PARTITION:]], Laser_points[idx[PARTITION:]]

    Laser_T_RobotBase_alternate_method = register(RobotBase_points_train, Laser_points_train)
    error = compute_errors(RobotBase_points_test, Laser_points_test, Laser_T_RobotBase_alternate_method)

    error_mean.append(error.mean())
    error_max.append(error.max())

print("Alternate Method:")
print(f"Mean: {np.mean(error_mean):0.2f}mm\t Max: {np.mean(error_max):0.2f}mm")

Alternate Method:
Mean: 0.89mm	 Max: 1.62mm
